# Исследование надежности заемщиков


## Знакомство с данными


Импортировали библиотеку pandas. Считали данные из csv-файла в датафрейм:


In [ ]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

Вывели первые 20 строчек датафрейма на экран:

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывели основную информацию о датафрейме:

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Вывели количество пропущенных значений для каждого столбца:

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработали на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом:

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработали значения в этом столбце: заменили все отрицательные значения положительными с помощью метода `abs()`:

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости вывели медианное значение трудового стажа `days_employed` в днях:

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставили их как есть. Тем более этот столбец не понадобится нам для исследования.

Вывели перечень уникальных значений столбца `children`:

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалили строки, в которых встречаются такие аномальные значения из датафрейма:

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз вывели перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены:

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполнили пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`:

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедились, что все пропуски заполнены:

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменили вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`:

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработали неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Привели их к нижнему регистру:

In [ ]:
data['education'] = data['education'].str.lower()

Вывели на экран количество строк-дубликатов в данных и удалили их:

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создали в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Вывели на экран перечень уникальных целей взятия кредита из столбца `purpose`:

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создали функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` появится строка `'операции с автомобилем'`.


In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


### Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

В первую очередь посмотрели на количество людей с не возвращенными в срок кредитами.
Выборку сразу разбили на группы по количеству детей, так как именно зависимость от их количества нам интересна.
Но количество задолжностей в каждой группе - это абсолютная величина, сравнивать её будет некорректо, поэтому нашли общее количество людей в каждой группе по количеству детей и посчитали долю людей с просроченными кредитами.
Для упрощения построения таблиц в дальнейшем написали функцию составления сводной таблицы для использования в следующих вопросах.

In [ ]:
def fin_table(data, index, new_index_name, st_values='debt', st_aggfunc=['count', 'sum', 'mean'], st_sort_by=('mean', 'debt')):
    '''
    Функция принимает датасет и столбец, по которому нужно построить сводную таблицу.
    Параметры по умолчанию: значения по которому будут производиться расчеты - есть ли задолжность,
                            расчеты: общее число клиентов, число клиентов с задолжностью и доля клиентов с задолжностью
                            столбец по которому будет отсортированна сводная таблица - доля клиентов с задолжностью
    Функция возвращает сводную таблицу по заданному датасету и выбранному столбцу с округлённой долей клиентов с задолжностью

    '''
    pivot = data.pivot_table(index=index, values=st_values, aggfunc=st_aggfunc).sort_values(by=st_sort_by)

    # Переименовали название индексов
    pivot.columns = ['Общее число клиентов', 'Число клиентов с задолжностью', 'Доля клиентов с задолжностью']
    pivot.index.names = [f'{new_index_name}']
    pivot = round(pivot, 3)
    return pivot

In [ ]:
children_pivot = fin_table(data, 'children', 'Количество детей')
children_pivot = children_pivot[1:]
children_pivot


,Общее число клиентов,Число клиентов с задолжностью,Доля клиентов с задолжностью
Количество детей,,,
0,14091,1063,0.075
3,330,27,0.082
1,4808,444,0.092
2,2052,194,0.095
4,41,4,0.098


**Вывод:**
- Люди с 5-ю детьми не имеют случаев задолжности (выбивающиеся показатели), также людей в данной группе очень мало (всего 9), поэтому эту группу убрали из оценки.
- Зависимость между количеством детей и возвратом кредита в срок мы нашли:
    1. Люди без детей реже всего имеют задолжности (**7.5%**).
    2. В группах людей имеющих детей дол невернувших кредит в срок несильно разняться (**9.2% - 9.8%**), за исключением группы людей с 3-мя детьми (**8,2%**).
- Интересной особенностью является также невысокая доля просрочивших возврат кредита людей с 3 детьми. Однако статистика по группам родителей 3-х детей и более непоказательная по сравнению с другими ввиду малочиленности этих групп.
   

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_pivot = fin_table(data, index='family_status', new_index_name='Семейное положение')
family_status_pivot


,Общее число клиентов,Число клиентов с задолжностью,Доля клиентов с задолжностью
Семейное положение,,,
вдовец / вдова,951,63,0.066
в разводе,1189,84,0.071
женат / замужем,12261,927,0.076
гражданский брак,4134,385,0.093
Не женат / не замужем,2796,273,0.098


**Вывод:**

Зависимость между семейным положением и возвратом кредита в срок есть:
1. Реже всего имеют задолжность по кредитам люди из группы *вдовец/вдова* (**6.6%**), также небольшая доля невернувших кредит в срок в группах *в разводе* (**7.1%**) и *женат/замужем* (**7.6%**).
2. Чаще задолжность можно встретить в группах *гражданский брак* (**9.3%**) и *не женат/не замужем* (**9.8%**).

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_pivot = fin_table(data, index='total_income_category', new_index_name='Категория дохода')
income_pivot

,Общее число клиентов,Число клиентов с задолжностью,Доля клиентов с задолжностью
Категория дохода,,,
D,349,21,0.060
B,5014,354,0.071
A,25,2,0.080
C,15921,1353,0.085
E,22,2,0.091


**Вывод:**
- Несколько групп по доходу весьма малочисленны, по сравнению с другими, а именно *группы A* и *E* - 25 и 22 человека соответсвенно. Даже в *группе D* мало людей - 349 человек, что более чем в 45 раз меньше, чем в *группе С*. Использование сильно малочисленных групп может дать непоказательный результат, поэтому для оценки использовались сопоставимые по количеству группы С и В.
- Зависимость между уровнем дохода и возвратом кредита в срок в группах существует, хоть и небольшая: в *группе В* доля клиентов с задолжностями составляет **7.1%**, а в *группе С* **8.5%**, то есть, чем выше доход, тем выше вероятность, что клиент вернёт кредит в срок.
- Интересные моменты из групп, которые не учитывались в общей оценке:
    1. Хотя в *группе А* всего 25 человек, доля клиентов в этой группе с задолжностью составляет **8%**, что больше, чем в *группе В*.
    2. В *группе Е* также очень мало людей, всего 22, но всё равно можно было бы ожидать большой доли клиентов с задолжностью, как в группе с самым низким доходом, однако разница не так и велика - **9.1%**.
    3. В *группе D* самая маленькая доля клиентов с задолжностью, всего **6%**.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_pivot = fin_table(data, index='purpose_category', new_index_name='Цель получения кредита')
purpose_pivot

,Общее число клиентов,Число клиентов с задолжностью,Доля клиентов с задолжностью
Цель получения кредита,,,
операции с недвижимостью,10751,780,0.073
проведение свадьбы,2313,183,0.079
получение образования,3988,369,0.093
операции с автомобилем,4279,400,0.093


**Вывод:**

Разные цели кредита по разному влияют на возврат кредита в срок:
1. Невысокая доля клиентов имеют задолжности по кредитам среди людей берущих кредиты на *операции с недвижимостью* (**7.3%**) и на *проведение свадьбы* (**7.9%**).
2. Чаще задолжность можно встретить среди клиентов берущих кредиты на *получение образования* и на *операции с автомобилем* (**9.3%** в обоих случаях).

#### 3.5 Возможные причины появления пропусков в исходных данных

Причина появления пропусков может крыться в техническом либо в человеческом факторе.

Пустых ячеек с количеством рабочих дней и пустых ячеек с общим доходом одинаковое количество, поэтому есть основания полагать, что это технический сбой. Например, на этапе загрузки и выгрузки данных. Код для проверки:
    
    len(data.loc[(data['days_employed'].isna()) & (data['total_income'].isna())])


Также, возможно, причиной является человеческий фактор: возможно у данных клиентов просто не собирали информацию, из-за чего появились пропуски. Клиенты сами могли не предоставить именно эту информацию.

### Общий вывод

- Мы провели работу над данными, для их дальнейшего анализа, а именно:
    1. Заполнили пропущенные данные.
    2. Удалили или преобразовали некорректные данные.
    3. Все данные привели к виду, подлежащему обработке.
    4. Удалили все дубликаты.
    5. Категоризировали данные для анализа.
- **Рекомендация**: установить причину появления пропусков в данных, возможные причины описаны выше.


- Мы проверили наличие зависимости возвращения кредитов в срок от разных критериев клентов банка:
    1. Клиенты без детей реже имеют задолжности по кредитам.
    2. Клиенты, состоявшие в браке (и те, кто в нём состоят) чаще возвращают кредиты в срок, чем люди не состоявшие в браке.
    3. Люди с большим доходом чаще возвращают кредиты в срок, чем люди с меньшим доходом.
    4. Кредиты выданные на операции с недвижимостью или на проведение свадьбы чаще возвращаются в срок, чем кредиты на образование и кредиты на операции с автомобилем.
- Мы выдвинули предположение о причине появления пропусков:
Причина пропусков - техническая, так как количество пропусков в двух колонках идентичное.
- Мы объяснили причину использования медианы для заполнения пропусков:
Медиана более предпочтительна для заполнения пропусков, из-за возможного наличия выбросов в наборе данных.


- **Рекомендация**: в итоге анализа, можно описать наилучшего и наихудшего кандидата для выдачи кредита:
    1. Идеальным кандидатом для выдачи кредита будет клиент не имеющий детей, но состоящий(или состоявший ранее) в браке, с доходом более 200000 руб. и берущий кредит на цели, связанные с недвижимостью, или на проведение свадьбы.
    2. Наиболее рискованным кандидатом для выдачи кредита будет клиент с 2-мя детьми и более, не состоящий в браке или состоящий в гражданском браке, с доходом менее 200000 руб. и берущий кредит на образование или на цели, связанные с автомобилем.